In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, train_test_split

%matplotlib inline

In [2]:
print(keras.__version__)

2.5.0


# Generate Kuzushiji images

### Create z-vector to sample digit from latent space

First, we need to define the region of latent space in which we are sampling images for our digit. The construct_numvec function creates the z-vector that specifies this region for either the latent space for KMNIST or the latent space for K49. Additionally, it appends a one-hot encoding of the class label of the kuzushiji character we want to generate. For example, if we have a z-vector [0, 1, 1, 1, 0, 1, 0, 1, 0, 0] and we want to generate the third character from the KMNIST data set, the one-hot encoding of the third character would be [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], and that would be concatenated to the z-vector.

In [3]:
def construct_numvec(digit: int, latent_dim: int=10, classes: int=10, z: list=None) -> list:
    """
        digit: int -> the class of the kuzushiji character to be generated
        latent_dim: int -> the number of dimensions in the latent space: equal to the number of classes
        classes: int -> the number of class labels for a given model (10 for KMNIST, 49 for K49)
        z: list -> the z-vector indicating the position in latent space to sample an character
    """
    
    out = np.zeros((1, latent_dim + classes))
    out[:, digit + latent_dim] = 1.
    if z is None:
        return(out)
    else:
        for i in range(len(z)):
            out[:,i] = z[i]
        return(out)

Next, we need to define a function that generates images for a specified digit. This function will receive arguments to define the number of rows and cols in the latent space we want sample from. Essentially, the number of images generated will be rows * cols.

In [4]:
def generate_images(digit: int=0, rows: int=10, cols: int=10, max_z: float=1.5, decoder=0, save_fig: bool=False) -> np.ndarray:
    """ Return an array
    
            digit: int -> the class of the kuzushiji character to be generated
            rows: int -> the width of the latent space
            cols: int -> the height of the latent space
            max_z: float ->
            decoder: int -> the generator to be used to create augmented data (0 for K49, 1 for KMNIST)
            save_fig: bool -> determines if an image of the augmented data should be created
    """
    
    images = []
    
    if decoder:
        decoder = keras.models.load_model("decoder-KMNIST")
        latent_dim = 10
        classes = 10
    else:
        decoder = keras.models.load_model("decoder-K49")
        latent_dim = 49
        classes = 49

    if save_fig:
        fig, axs = plt.subplots(rows, cols)
        fig.set_figwidth(28)
        fig.set_figheight(28)

    for i in range(0, rows):
        z1 = (((i / (rows-1)) * max_z)*2) - max_z
        for j in range(0, cols):
            z2 = (((j / (cols-1)) * max_z)*2) - max_z
            z_ = [z1, z2]
            vec = construct_numvec(digit, latent_dim, classes, z_)
            decoded = decoder.predict(vec)
            images.append(decoded.reshape(28,28))
            if save_fig:
                axs[i, j].imshow(decoded.reshape(28,28), cmap=plt.cm.gray)
                axs[i, j].axis('off')
    
    if save_fig:
        fig.savefig("image_%d_%d_%d.png" % (digit, rows, cols))
        
    return np.array(images)

In [5]:
# Generate some example images of the first class for KMNIST
aug_data = generate_images(digit=0, rows=10, cols=10, max_z=1.5, decoder=1, save_fig=False)

In [6]:
aug_data.shape

(100, 28, 28)

### Generate images from latent space for KMNIST or K49

Let's generate some augmented data to train with. We'll start off with creating 100 sample images from each class of KMNIST. Feel free to change the number of images generated by altering the number of rows and columns in the generate_aug_kmnist() function.

In [16]:
def generate_aug_kmnist(rows=10, cols=10):
    images = generate_images(digit=0, rows=rows, cols=cols, max_z=1.5, decoder=1)
    labels = []
    labels.append([0 for x in range(rows) for y in range(cols)])
    for digit in range(1,10):
        aug_data = generate_images(digit=digit, rows=rows, cols=cols, max_z=1.5, decoder=1)
        images = np.concatenate((images, aug_data), axis=0)
        labels.append([digit for x in range(rows) for y in range(cols)])
        
    return images, np.array(labels).flatten()

    
def generate_aug_k49(rows=10, cols=10):
    images = generate_images(digit=0, rows=rows, cols=cols, max_z=1.5, decoder=0)
    labels = []
    labels.append([0 for x in range(rows) for y in range(cols)])
    for digit in range(1,49):
        aug_data = generate_images(digit=digit, rows=rows, cols=cols, max_z=1.5, decoder=0)
        images = np.concatenate((images, aug_data), axis=0)
        labels.append([digit for x in range(cols) for y in range(cols)])
        
    return images, np.array(labels).flatten()

When training the model later, make sure that if you desire to train using the K49 data set, comment out the line for generating KMNIST and uncomment for generating K49. The same applies for the KMNIST data. NOTE: it will take a few minutes to generate the augmented data for the K49 data set due to the large number of class labels.

In [17]:
aug_kmnist_data, aug_kmnist_labels = generate_aug_kmnist()
# aug_kmnist_data, aug_kmnist_labels = generate_aug_k49()

### Define CNN model and set up training

Let's define the CNN model we'll use to evaluate the quality of the augmented data. The functions below define the CNN model, the training process, and evaluation process.

In [18]:
# define cnn model
def define_model(kmnist=True):
    """
        kmnist: bool -> specifies whether the CNN model to be constructed should be for KMNIST(True) or K49(False)
    """
    model = keras.models.Sequential()
    
    if kmnist:
        model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
        model.add(keras.layers.MaxPooling2D((2, 2)))
        model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
        model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
        model.add(keras.layers.MaxPooling2D((2, 2)))
        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(100, activation='relu', kernel_initializer='he_uniform'))
        model.add(keras.layers.Dense(10, activation='softmax'))
        
    else:
        model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
        model.add(keras.layers.MaxPooling2D((2, 2)))
        model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
        model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
        model.add(keras.layers.MaxPooling2D((2, 2)))
        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(100, activation='relu', kernel_initializer='he_uniform'))
        model.add(keras.layers.Dense(49, activation='softmax'))
        
    # compile model
    opt = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

 
# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, X_test, epochs=10, n_folds=1, kmnist=True):
    scores, histories, predictions = list(), list(), list()
    callback = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=5)
    
    # If n_folds is less than 2, do not perform cross-validation
    # Otherwise, implement cross-validation
    if n_folds > 1:
        # prepare cross validation
        kfold = KFold(n_folds, shuffle=True, random_state=1)
        i = 1
        for train_ix, test_ix in kfold.split(dataX):
            print(len(train_ix), len(test_ix))
            print("- - - - -\nFold %d\n- - - - -" % i)
            # define model
            model = define_model(kmnist)
            # select rows for train and test
            trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]

            # fit model
            history = model.fit(trainX, trainY, epochs=epochs, batch_size=128, validation_data=(testX, testY), callbacks=[callback], verbose=1)
            # evaluate model
            _, acc = model.evaluate(testX, testY, verbose=0)
            print('> %.3f\n' % (acc * 100.0))
            prediction = model.predict(X_test)

            # Save validation loss, accuracies, and predictions
            scores.append(acc)
            histories.append(history)
            predictions.append(prediction)

            i += 1

        return scores, histories, predictions
    
    # define model
    model = define_model(kmnist)
    # select rows for train and test
    trainX, testX, trainY, testY = train_test_split(dataX, dataY, test_size=0.20, random_state=42)

    # fit model
    history = model.fit(trainX, trainY, epochs=epochs, batch_size=128, validation_data=(testX, testY), callbacks=[callback], verbose=1)
    # evaluate model
    _, acc = model.evaluate(testX, testY, verbose=0)
    print('> %.3f\n' % (acc * 100.0))
    prediction = model.predict(X_test)

    # Save validation loss, accuracies, and predictions
    scores.append(acc)
    histories.append(history)
    predictions.append(prediction)
    
    return scores, histories, predictions

    
# Display graph for validation loss
def summarize_diagnostics(histories):
    for i in range(len(histories)):
        # plot loss
        plt.subplot(2, 1, 1)
        plt.title('Cross Entropy Loss')
        plt.plot(histories[i].history['loss'], color='blue', label='train')
        plt.plot(histories[i].history['val_loss'], color='orange', label='test')
        # plot accuracy
        plt.subplot(2, 1, 2)
        plt.title('Classification Accuracy')
        plt.plot(histories[i].history['accuracy'], color='blue', label='train')
        plt.plot(histories[i].history['val_accuracy'], color='orange', label='test')
    plt.show()
    

# Display boxplot for accuracies
def summarize_performance(scores):
    # print summary
    print('Accuracy: mean=%.3f std=%.3f, n=%d' % (np.mean(scores)*100, np.std(scores)*100, len(scores)))
    # box and whisker plots of results
    plt.boxplot(scores)
    plt.show()
    

# Train and evaluate the model(s)
def run_test_harness(trainX, trainY, testX, testY, epochs=10, n_folds=1, kmnist=True):
    # evaluate model
    scores, histories, predictions = evaluate_model(trainX, trainY, testX, epochs=epochs, n_folds=n_folds, kmnist=kmnist)
    
    if n_folds > 1:
        # learning curves
        summarize_diagnostics(histories)
        # summarize estimated performance
        summarize_performance(scores)

### Load Kuzushiji data set and pre-process

Before we train, we need to load our data set that will be used to train the CNN model. For now, we'll start with using KMNIST. When we load the KMNIST data set, we will normalize the original and augmented data and then concatenate them together to form the new training set. 

In [19]:
# scale pixels
def prep_pixels(train, *args):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = None
    if len(args) == 1:
        test_norm = args[0].astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    if len(args) == 1:
        test_norm = test_norm / 255.0
    # return normalized images
    return train_norm, test_norm

# load train and test dataset
def load_dataset(kmnist=True):
    
    if kmnist:
        # Load the KMNIST data set
        training_data_files = "./dataset/kmnist-train-imgs.npz"
        testing_data_files = "./dataset/kmnist-test-imgs.npz"
        training_label_files = "./dataset/kmnist-train-labels.npz"
        testing_label_files = "./dataset/kmnist-test-labels.npz"
        
        (trainX, trainY) = np.load(training_data_files)["arr_0"], np.load(training_label_files)["arr_0"]
        (testX, testY) = np.load(testing_data_files)["arr_0"], np.load(testing_label_files)["arr_0"]

        # Reshape the images for training 
        trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
        testX = testX.reshape((testX.shape[0], 28, 28, 1))
        
        # Convert class labels into one-hot encoding vectors
        trainY, testY = keras.utils.to_categorical(trainY, num_classes=10), keras.utils.to_categorical(testY, num_classes=10)
        
        # prepare pixel data
        trainX, testX = prep_pixels(trainX, testX)
        
        # Concatenate original KMNIST data with augmented data
        trainX = np.concatenate((trainX, aug_kmnist_data), axis=0)
        trainY = np.concatenate((trainY, aug_kmnist_labels_1h), axis=0)

        return trainX, trainY, testX, testY
    
    # Load the K49 data set
    training_data_files = "./dataset/k49-train-imgs.npz"
    testing_data_files = "./dataset/k49-test-imgs.npz"
    training_label_files = "./dataset/k49-train-labels.npz"
    testing_label_files = "./dataset/k49-test-labels.npz"
    
    (trainX, trainY) = np.load(training_data_files)["arr_0"], np.load(training_label_files)["arr_0"]
    (testX, testY) = np.load(testing_data_files)["arr_0"], np.load(testing_label_files)["arr_0"]
    
    # Reshape the images for training 
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))

    # Convert class labels into one-hot encoding vectors
    trainY, testY = keras.utils.to_categorical(trainY, num_classes=49), keras.utils.to_categorical(testY, num_classes=49)
    
    # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)
    
    # Concatenate original K49 data with augmented data
    trainX = np.concatenate((trainX, aug_kmnist_data), axis=0)
    trainY = np.concatenate((trainY, aug_kmnist_labels_1h), axis=0)
    
    return trainX, trainY, testX, testY


In [20]:
aug_kmnist_data, _ = prep_pixels(aug_kmnist_data)
aug_kmnist_data = aug_kmnist_data.reshape((aug_kmnist_data.shape[0], 28, 28, 1))

In [21]:
unique = np.unique(aug_kmnist_labels)
aug_kmnist_labels_1h = keras.utils.to_categorical(aug_kmnist_labels, num_classes=len(unique))

In [22]:
# load dataset
trainX, trainY, testX, testY = load_dataset(kmnist=True)

### Run Demo

Now it's time to run! Run the function below and observe the progress of the CNN models as they train with the augmented data. If you want to train with the K49 data set, make sure to switch the kmnist argument from True to False, and remember to make changes to the code above as well to load the K49 data set. This means changing the value of the kmnist argument from True to False in the load_dataset() function above as well as calling the generate_aug_k49() function instead of generate_aug_kmnist().

In [23]:
run_test_harness(trainX, trainY, testX, testY, epochs=5, n_folds=1, kmnist=True)

Epoch 1/5
382/382 [==============================] - 21s 54ms/step - loss: 0.4768 - accuracy: 0.8476 - val_loss: 0.2241 - val_accuracy: 0.9280
Epoch 2/5
382/382 [==============================] - 20s 52ms/step - loss: 0.1845 - accuracy: 0.9407 - val_loss: 0.1651 - val_accuracy: 0.9474
Epoch 3/5
382/382 [==============================] - 20s 53ms/step - loss: 0.1355 - accuracy: 0.9553 - val_loss: 0.1496 - val_accuracy: 0.9502
Epoch 4/5
382/382 [==============================] - 20s 53ms/step - loss: 0.1083 - accuracy: 0.9642 - val_loss: 0.1327 - val_accuracy: 0.9576
Epoch 5/5
382/382 [==============================] - 20s 53ms/step - loss: 0.0940 - accuracy: 0.9684 - val_loss: 0.1312 - val_accuracy: 0.9593
> 95.926

